In [51]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

In [52]:
df = pd.read_csv('./MIMICIV/data_20221229_timeseries.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75199 entries, 0 to 75198
Data columns (total 43 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   subject_id            75199 non-null  int64  
 1   stay_id               75199 non-null  int64  
 2   to_date               74886 non-null  object 
 3   Platelets x1000       71186 non-null  float64
 4   WBC x1000             71361 non-null  float64
 5   Hgb                   71301 non-null  float64
 6   Albumin               13728 non-null  float64
 7   Total Protein         517 non-null    float64
 8   Total Bilirubin       25770 non-null  float64
 9   PO2                   42459 non-null  float64
 10  PaCO2                 42452 non-null  float64
 11  Glucose               71217 non-null  float64
 12  BUN                   71685 non-null  float64
 13  pH                    48132 non-null  float64
 14  Sodium                71676 non-null  float64
 15  Potassium          

In [3]:
# df.query(expr="stay_id == @temp")

In [53]:
df.sort_values(['to_date'],ascending=True) #對'to_date'進行排序_小->大

,subject_id,stay_id,to_date,Platelets x1000,WBC x1000,Hgb,Albumin,Total Protein,Total Bilirubin,PO2,...,mbp_ni,Compliance,FiO2,Minute Ventilation,Mean Airway Pressure,Peak Airway Pressure,PEEP,Plateau,Tidal Volume,Respiratory Rate
54530,17282717,39473385,2110-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,98.526316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.315789
54531,17282717,39473385,2110-03-01,193.0,7.1,35.6,3.3,NaN,0.7,NaN,...,111.583333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.708333
54532,17282717,39473385,2110-03-02,217.0,61.0,34.0,NaN,NaN,NaN,NaN,...,100.916667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.375000
54533,17282717,39473385,2110-03-03,186.0,14.9,35.0,NaN,NaN,NaN,208.0,...,89.714286,NaN,76.0,NaN,NaN,NaN,NaN,NaN,NaN,21.958333
54534,17282717,39473385,2110-03-04,160.0,20.0,34.8,NaN,NaN,0.8,NaN,...,80.958333,NaN,50.0,NaN,NaN,NaN,NaN,NaN,NaN,20.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73981,19855614,30240922,NaN,NaN,NaN,15.6,NaN,NaN,NaN,444.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74593,19930120,30461247,NaN,289.0,14.4,33.5,NaN,NaN,NaN,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74594,19930120,30461247,NaN,265.0,14.5,33.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74595,19930120,30461247,NaN,248.0,10.7,34.8,NaN,NaN,NaN,411.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
df = df.drop(['Albumin', 'Total Protein','Compliance'], axis=1) #<70% drop

In [55]:
df

,subject_id,stay_id,to_date,Platelets x1000,WBC x1000,Hgb,Total Bilirubin,PO2,PaCO2,Glucose,...,dbp_ni,mbp_ni,FiO2,Minute Ventilation,Mean Airway Pressure,Peak Airway Pressure,PEEP,Plateau,Tidal Volume,Respiratory Rate
0,10001884,37510196,2131-01-11,171.0,18.4,31.7,0.4,49.0,69.0,148.0,...,72.761905,85.761905,58.333333,9.900000,11.360000,28.400000,5.033333,23.0,379.750000,18.000000
1,10001884,37510196,2131-01-12,149.0,12.0,31.0,0.2,65.0,63.0,145.0,...,78.937500,94.781250,44.285714,9.444444,8.350000,18.166667,5.000000,NaN,369.833333,20.184211
2,10001884,37510196,2131-01-13,144.0,21.0,30.5,NaN,69.0,53.0,199.0,...,76.958333,96.166667,40.000000,10.261111,9.550000,22.333333,5.090909,17.4,418.384615,20.500000
3,10001884,37510196,2131-01-14,119.0,12.6,31.6,0.4,91.0,61.0,165.0,...,82.909091,98.727273,40.000000,9.441176,10.042857,23.428571,5.000000,20.0,410.428571,20.724138
4,10001884,37510196,2131-01-15,109.0,12.1,31.4,0.5,NaN,NaN,170.0,...,77.541667,92.750000,40.000000,9.561111,10.566667,23.500000,5.000000,NaN,341.333333,20.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75194,19998591,36794489,2185-07-08,188.0,7.0,37.7,NaN,NaN,NaN,126.0,...,59.791667,68.125000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.625000
75195,19998591,36794489,2185-07-09,148.0,6.2,38.1,NaN,NaN,NaN,113.0,...,57.285714,65.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.500000
75196,19998591,36794489,2185-07-10,157.0,4.7,38.7,NaN,NaN,NaN,143.0,...,49.304348,75.035714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.718750
75197,19998591,36794489,2185-07-11,163.0,5.1,38.2,NaN,NaN,NaN,90.0,...,49.054054,59.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.564103


In [56]:
df.interpolate(method='linear',limit_direction='both', inplace=True) #進行插值(內插)

In [57]:
df.interpolate(method='linear',limit_direction='forward', inplace=True)

In [58]:
df.interpolate(method='linear',limit_direction='backward', inplace=True)

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75199 entries, 0 to 75198
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   subject_id            75199 non-null  int64  
 1   stay_id               75199 non-null  int64  
 2   to_date               74886 non-null  object 
 3   Platelets x1000       75199 non-null  float64
 4   WBC x1000             75199 non-null  float64
 5   Hgb                   75199 non-null  float64
 6   Total Bilirubin       75199 non-null  float64
 7   PO2                   75199 non-null  float64
 8   PaCO2                 75199 non-null  float64
 9   Glucose               75199 non-null  float64
 10  BUN                   75199 non-null  float64
 11  pH                    75199 non-null  float64
 12  Sodium                75199 non-null  float64
 13  Potassium             75199 non-null  float64
 14  Magnesium             75199 non-null  float64
 15  Calcium            

In [61]:
subject_id = df['subject_id'].unique()

In [62]:
len(subject_id)

5165

In [63]:
subject_id

array([10001884, 10002348, 10003400, ..., 19995595, 19997367, 19998591],
      dtype=int64)

In [65]:
df.head(5)

,subject_id,stay_id,to_date,Platelets x1000,WBC x1000,Hgb,Total Bilirubin,PO2,PaCO2,Glucose,...,dbp_ni,mbp_ni,FiO2,Minute Ventilation,Mean Airway Pressure,Peak Airway Pressure,PEEP,Plateau,Tidal Volume,Respiratory Rate
0,10001884,37510196,2131-01-11,171.0,18.4,31.7,0.4,49.00,69.000000,148.0,...,72.761905,85.761905,58.333333,9.900000,11.360000,28.400000,5.033333,23.00000,379.750000,18.000000
1,10001884,37510196,2131-01-12,149.0,12.0,31.0,0.2,65.00,63.000000,145.0,...,78.937500,94.781250,44.285714,9.444444,8.350000,18.166667,5.000000,20.20000,369.833333,20.184211
2,10001884,37510196,2131-01-13,144.0,21.0,30.5,0.3,69.00,53.000000,199.0,...,76.958333,96.166667,40.000000,10.261111,9.550000,22.333333,5.090909,17.40000,418.384615,20.500000
3,10001884,37510196,2131-01-14,119.0,12.6,31.6,0.4,91.00,61.000000,165.0,...,82.909091,98.727273,40.000000,9.441176,10.042857,23.428571,5.000000,20.00000,410.428571,20.724138
4,10001884,37510196,2131-01-15,109.0,12.1,31.4,0.5,91.75,60.083333,170.0,...,77.541667,92.750000,40.000000,9.561111,10.566667,23.500000,5.000000,20.22807,341.333333,20.100000


In [66]:
N = 7
timeseries_Data = []

In [67]:
for sub_id in subject_id:
    sub_df = df[df['subject_id'] == sub_id]
    if sub_df.shape[0] < 7:
        continue
    day_df = sub_df[-N:]
    day_df = day_df.drop(['subject_id', 'stay_id','to_date'], axis=1)
    timeseries_Data.append(day_df.to_numpy())
    

In [68]:
day_df

,Platelets x1000,WBC x1000,Hgb,Total Bilirubin,PO2,PaCO2,Glucose,BUN,pH,Sodium,...,dbp_ni,mbp_ni,FiO2,Minute Ventilation,Mean Airway Pressure,Peak Airway Pressure,PEEP,Plateau,Tidal Volume,Respiratory Rate
75192,303.0,10.1,38.0,0.55,45.0,54.0,137.0,23.0,6.664286,147.0,...,61.880000,70.520000,50.0,8.98125,7.5,15.0,4.375,18.0,322.25,21.115385
75193,222.0,8.3,38.1,0.50,45.0,54.0,116.0,12.0,6.331429,142.0,...,54.125000,63.666667,50.0,8.98125,7.5,15.0,4.375,18.0,322.25,16.666667
75194,188.0,7.0,37.7,0.50,45.0,54.0,126.0,7.0,5.998571,139.0,...,59.791667,68.125000,50.0,8.98125,7.5,15.0,4.375,18.0,322.25,18.625000
75195,148.0,6.2,38.1,0.50,45.0,54.0,113.0,5.0,5.665714,136.0,...,57.285714,65.000000,50.0,8.98125,7.5,15.0,4.375,18.0,322.25,16.500000
75196,157.0,4.7,38.7,0.50,45.0,54.0,143.0,4.0,5.332857,138.0,...,49.304348,75.035714,50.0,8.98125,7.5,15.0,4.375,18.0,322.25,23.718750
75197,163.0,5.1,38.2,0.50,45.0,54.0,90.0,3.0,5.000000,139.0,...,49.054054,59.166667,50.0,8.98125,7.5,15.0,4.375,18.0,322.25,15.564103
75198,186.0,12.1,39.8,0.50,45.0,54.0,61.0,2.0,5.000000,141.0,...,54.571429,60.833333,50.0,8.98125,7.5,15.0,4.375,18.0,322.25,17.214286


In [69]:
timeseries_Datas = np.array(timeseries_Data)

In [70]:
timeseries_Datas[2]

array([[ 25.        ,   8.        ,  34.9       ,   4.7       ,
         88.33333333,  40.33333333, 141.        ,  16.        ,
          6.276     , 142.        ,   3.7       ,   1.9       ,
          8.2       , 107.        ,   0.6       ,  31.        ,
          1.9       , 110.        ,  50.        ,  35.        ,
         15.4       ,  95.16666667, 102.33333333, 106.79166667,
         63.75      ,  72.41666667, 106.79166667,  63.75      ,
         72.41666667,  61.5       ,   9.17222222,   7.33333333,
         14.66666667,   5.33333333,  23.33333333, 488.55555556,
         23.875     ],
       [ 41.        ,   9.5       ,  34.6       ,   3.4       ,
        101.91666667,  42.41666667, 143.        ,  17.        ,
          6.414     , 142.        ,   3.9       ,   1.8       ,
          8.1       , 108.        ,   0.6       ,  32.        ,
          2.3       , 120.        ,  48.        ,  29.        ,
         15.7       ,  95.57142857, 112.34782609, 100.42857143,
         58.57142

In [72]:
timeseries_Datas.shape

(5159, 7, 37)

In [73]:
np.save('timeseries_Datas.npy', timeseries_Datas)

In [74]:
#static_Y
static_df = pd.read_csv('./MIMICIV/data_20230101_static.csv')
static_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5158 entries, 0 to 5157
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   subject_id         5158 non-null   int64  
 1   stay_id            5158 non-null   int64  
 2   dod                2361 non-null   object 
 3   gender             5158 non-null   object 
 4   admission_age      5158 non-null   int64  
 5   race               5158 non-null   object 
 6   Troponin - I       0 non-null      float64
 7   Troponin - T       5158 non-null   float64
 8   Ferritin           5158 non-null   float64
 9   Transferrin        5158 non-null   float64
 10  CRP                5158 non-null   float64
 11  Fibrinogen         5158 non-null   float64
 12  HDL                5158 non-null   float64
 13  LDL                5158 non-null   float64
 14  Total Cholesterol  5158 non-null   float64
 15  Direct Bilirubin   5158 non-null   float64
 16  Lactate            5158 

In [83]:
Y = []
static_df.shape[0]

5158

In [95]:
# static_df['dod'].isna().astype(np.int64).to_numpy()

array([0, 1, 0, ..., 1, 1, 0], dtype=int64)

In [96]:
np.save('dod_data.npy', static_df['dod'].isna().astype(np.int64).to_numpy())

In [100]:
stsatic_subject_id = static_df['subject_id'].unique()

In [101]:
stsatic_subject_id

array([10001884, 10002348, 10003400, ..., 19995595, 19997367, 19998591],
      dtype=int64)

In [99]:
for sub_id in subject_id:
    sub_df = df[df['subject_id'] == sub_id]

In [102]:
# N = 6 #取每個人最後7天住院資料
# timeseries_Data = []
# for i in range(len(subject_id)): #'subject_id'共有5165不同的
#     for j in range(interpolate_df.shape[0]): #shape:(橫列數,直行數) / [0]回傳列數
#         if subject_id[i] == interpolate_df['subject_id'][j] and subject_id[i] != interpolate_df['subject_id'][j+1]: #取每個'subject_id'的最後7筆data
#             timeseries_data = interpolate_df.iloc[j-N:j+1,]  #存入timeseries_data
#             timeseries_Data.append(timeseries_data.to_numpy())
#     #if i == 5:
#         #break